In [4]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np
import os
fmp_key = os.getenv("FMI_API_KEY")

In [7]:
# Ensure repo root is on sys.path so `common` can be imported
from pathlib import Path
import sys, os

# Find a parent directory that contains the 'common' folder
cwd = Path.cwd()
repo_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "common").exists():
        repo_root = p
        break

if repo_root is None:
    raise RuntimeError("Could not find 'common' folder in current directory or parents. Set PYTHONPATH or chdir to repo root.")

# Put repo root at front of sys.path
sys.path.insert(0, str(repo_root))
print("Added to sys.path:", str(repo_root))

# Now import and test
from common.PortConnect import Port_Connect
from common.Portfolio import Portfolio
# quick smoke test (optional)
pc = Port_Connect(api_key=os.getenv("API_KEY"))
print("Imported Port_Connect, instance:", type(pc))

Added to sys.path: /workspaces/APPLIED_FINANCE_ANALYTICS
Imported Port_Connect, instance: <class 'common.PortConnect.Port_Connect'>


# Use the portfolio connector to retrieve prices

In [9]:
port = Port_Connect(api_key=fmp_key)

# Asset-Only Asset Allocations and Mean–Variance Optimization

## Calculate the investor's utility function for the asset mix

This is a formula that calculates the expected return for the investor, as a function of the expected return and an inverse relationship of the investor's risk aversion and variance return

Lets use the S&P 500 for this calculation

In [11]:
url = (f"https://financialmodelingprep.com/stable/historical-price-eod/light?symbol=AAPL&apikey={fmp_key}")
df = port._get_df(url=url,is_historical=True)
df


,symbol,price,volume
date,,,
2020-12-07,AAPL,123.7500,86712000
2020-12-08,AAPL,124.3800,82225512
2020-12-09,AAPL,121.7800,115089200
2020-12-10,AAPL,123.2400,81312200
2020-12-11,AAPL,122.4100,86939800
...,...,...,...
2025-12-01,AAPL,283.1000,46587722
2025-12-02,AAPL,286.1900,53669532
2025-12-03,AAPL,284.1500,43538700


In [ ]:
syk = port.get_closing_prices('SYK')
syk['Change'] = syk['SYK'].pct_change()
syk.sample(3)

### Temporarily Fix Code

In [ ]:
#!/usr/bin/env python
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url = (f"https://financialmodelingprep.com/stable/ratings-snapshot?symbol=AAPL&apikey={fmp_key}")
       
print(get_jsonparsed_data(url))

parsed_url = get_jsonparsed_data(url)

df = pd.DataFrame(parsed_url)

if "symbol" in df.columns:
    df.set_index('symbol',inplace=True)

df


In [ ]:
#!/usr/bin/env python
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url = (f"https://financialmodelingprep.com/stable/historical-price-eod/light?symbol=AAPL&apikey={fmp_key}")
print(get_jsonparsed_data(url))
parsed_url = get_jsonparsed_data(url)
stock_url = pd.DataFrame(parsed_url)
stock_url["date"] = pd.to_datetime(stock_url["date"])
stock_url.sort_values(by='date',ascending=True,inplace=True)
stock_url.set_index('date',inplace=True)
stock_url.set_index = pd.to_datetime(stock_url.index)
stock_url



In [ ]:
def historical_closing_price(ticker:str,interval:str = '1d'):
    url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?serietype=line&apikey={fmp_key}'

    url = f'"https://financialmodelingprep.com/stable/historical-chart/4hour?symbol={ticker}&apikey={fmp_key}"'
    
    df = self._get_df(url,True)
    
    if df is None:
        return None
        
    resampled_df = self.resample_prices(df=df,frequency=interval)

    resampled_df.set_index('date',inplace=True)

    return resampled_df

In [ ]:
historical_closing_price(ticker='AAPL')

### End of Temporarily Fix Code

In [ ]:
portfolio.annualize_rets(syk['Change'],252)

In [ ]:
ticker = 'BTC'

df = port.get_closing_prices(ticker)
df['Change'] = df[ticker].pct_change()
portfolio.annualize_rets(df['Change'],252)

In [ ]:
portfolio = 

Calculate the percentage change of the daily closing prices

In [ ]:
spy['Change']  = spy['SPY'].pct_change()
spy.tail(3)

What is the range of the data?

In [ ]:
print("The first date is ",spy.index[0])
print("The last date is ",spy.index[-1])

Now, let's calculate the mean return and variance of the whole series

In [ ]:
mean_return = portfolio.annualize_rets(spy['Change'],252)
mean_vol = spy['Change'].var() * 252

print('Mean return is ',np.round(mean_return,5) * 100)
print('Mean Variance is ',np.round(mean_vol,5) * 100)

Lets calculate the investors utility of the SPY

$$
U_m = E(R_m) - 0.5 \lambda \sigma_m^2
$$

$U_m$ = the investor’s utility for asset mix (allocation) m

$E(R_m)$  = the return for asset mix m

$ \lambda $ = the investor’s risk aversion coefficient

$ \sigma_m^2 $ = the expected variance of return for asset mix m

so lets assume that the risk aversion coefficient for an investor is 2 (Risk tolerant investor)

In [ ]:
ret = mean_return
vol = mean_vol
risk_coefficient = 2

investor_utility = ret - 0.5*risk_coefficient*vol
print('The investors utility is ',np.round(investor_utility,4) * 100)

so, what does it mean that the investor's utility is 4.87%? 

The 4.87% can be interpreted as the certainty equivalent return—the guaranteed return the investor would consider equally desirable as the risky investment. Despite the potential for higher returns with risk, the risk penalty “lowers” the effective return to 4.87% given the investor’s risk preferences.



## Use Monte Carlo Simulation in order to analyze a portfolio for retirement

The client has 1M USD as his initial portfolio, wants to withdraw 5OK USD per year and expects to live 25 years more. He wants to live his children the same 1M USD in real terms (inflation of 2.6% annually)

Assuming a return of 8.5% and a volatility of 18%, what is the probability of achieving the desired goal?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
initial_portfolio = 1_000_000     # Initial portfolio value in USD$
withdrawal = 50_000               # Annual withdrawal in USD$
years = 25                      # Retirement period (years)
n_simulations = 10000           # Number of Monte Carlo simulation runs

# Capital market expectations for the portfolio (nominal values)
nominal_return_mean = 0.085     # 7.5% expected nominal return per year
nominal_return_std = 0.18       # 11% standard deviation of nominal returns
inflation = 0.026               # 2.6% annual inflation

# Create an array to hold the portfolio values for each simulation over time.
# We include year 0 (the starting portfolio) and then one value per year.
portfolio_sim = np.zeros((n_simulations, years + 1))
portfolio_sim[:, 0] = initial_portfolio

# Run the Monte Carlo simulation for each simulation path
for sim in range(n_simulations):
    portfolio = initial_portfolio
    for year in range(1, years + 1):
        # Draw a random nominal return for the year from a normal distribution.
        r = np.random.normal(nominal_return_mean, nominal_return_std)
        # Update the portfolio: grow by the nominal return, then withdraw the fixed amount.
        portfolio = portfolio * (1 + r) - withdrawal
        portfolio_sim[sim, year] = portfolio

# Discount nominal portfolio values to real terms by adjusting for inflation.
# Create an array of discount factors: (1 + inflation) ** year
years_array = np.arange(years + 1)
discount_factors = (1 + inflation) ** years_array

# Divide each year's nominal value by the corresponding discount factor.
portfolio_sim_real = portfolio_sim / discount_factors[np.newaxis, :]

# Compute selected percentiles (10th, 25th, 50th, 75th, and 90th) across all simulations for each year.
percentiles = [10, 25, 50, 75, 90]
portfolio_percentiles = np.percentile(portfolio_sim_real, percentiles, axis=0)

# Plot the percentile paths over time.
plt.figure(figsize=(10, 6))
for i, perc in enumerate(percentiles):
    plt.plot(years_array, portfolio_percentiles[i, :], label=f"{perc}th percentile")
plt.xlabel("Year")
plt.ylabel("Real Portfolio Value (USD$)")
plt.title("Monte Carlo Simulation: Real Portfolio Value Over 25 Years")
plt.legend()
plt.grid(True)
plt.show()

# Print the median (50th percentile) real portfolio value at the end of 25 years.
median_bequest = np.median(portfolio_sim_real[:, -1])
print("Median Real Portfolio Value at end of 25 years: USD$", round(median_bequest, 2))


# Create Portfolio

In [ ]:
prices = port.get_closing_prices(['MSFT','AAPL','KO','AMZN','O','AGNC','NVDA','FB'])
prices = prices[prices.index > '2000-01-01']
prices.sample(2)

In [ ]:
rets = prices.pct_change()
cov_matrix = rets.cov() * 252
cov_matrix

In [ ]:
port_stats = Portfolio_Stats()
port_stats.gmv(cov_matrix)

In [ ]:
rets

In [ ]:
port_rets = port_stats.annualize_rets(rets,252)
port_rets

In [ ]:
rets.cov() * 252

In [ ]:
### Minimum covariance matrix
returns = port_stats.annualize_rets(rets,252)
minimum_variance_weights = port_stats.gmv(cov_matrix)
print(minimum_variance_weights)
portfolio_return = port_stats.portfolio_return(minimum_variance_weights,returns)
portfolio_vol = port_stats.portfolio_vol(minimum_variance_weights,cov_matrix)
print(portfolio_return)
print(portfolio_vol)

In [ ]:
### Maximize Sharp Ratio
returns = port_stats.annualize_rets(rets,252)
maximum_sharpe_weights = port_stats.msr(0.03,returns,cov_matrix)
print(np.round(maximum_sharpe_weights,5))
portfolio_return = port_stats.portfolio_return(maximum_sharpe_weights,returns)
portfolio_vol = port_stats.portfolio_vol(maximum_sharpe_weights,cov_matrix)
print(portfolio_return)
print(portfolio_vol)

In [ ]:
port_stats.plot_ef(10,returns,cov_matrix,show_ew=True, show_gmv=True,show_cml=True,riskfree_rate=0.05,legend=True)

# Black-Litterman Model
### Use the BL Model to better forecast asset returns and asset allocations

# Liability Based Investing

## Surplus Approach

This example seeks to maximize the surplus—that is, the expected excess of asset returns over a given liability—while penalizing portfolio risk. We set up an objective function that is the negative of the risk‐adjusted surplus and then minimize it. The constraints require that weights sum to 1 and are nonnegative.

Explanation:
This model computes the portfolio weights that maximize the risk‐adjusted surplus (asset returns minus liability and risk penalty). The optimization is done using SciPy’s minimize with equality and bound constraints.

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

# Parameters for Surplus Optimization
mu = np.array([0.08, 0.12])   # Expected returns for two assets
Sigma = np.array([[0.1, 0.05],
                  [0.05, 0.2]])  # Covariance matrix of asset returns
liability = 0.10              # Liability (e.g., expected cash outflow)
risk_aversion = 1.0           # Risk aversion coefficient

# Objective function: maximize [mu.dot(w) - liability - 0.5 * risk_aversion * (w.T @ Sigma @ w)]
# We minimize the negative of the objective.
def surplus_objective(w):
    expected_surplus = np.dot(mu, w) - liability
    risk = np.dot(w, np.dot(Sigma, w))
    return - (expected_surplus - 0.5 * risk_aversion * risk)

# Constraints: sum of weights equals 1.
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
# Bounds: no short selling (weights between 0 and 1)
bounds = [(0, 1) for _ in range(len(mu))]

# Initial guess for weights
w0 = np.array([0.5, 0.5])

# Solve the optimization problem.
result = minimize(surplus_objective, w0, bounds=bounds, constraints=constraints)

print("Surplus Optimization Optimal Weights:")
print(result.x)

# For clarity, present the results in a DataFrame.
df_surplus = pd.DataFrame({
    'Asset': ['Asset 1', 'Asset 2'],
    'Weight': result.x
})
df_surplus


## Hedged Return Portfolio

In a hedged return portfolio, the objective is to generate attractive returns while neutralizing sensitivity to liability-related risks. Here, each instrument has an associated beta relative to a liability factor. We maximize the expected return while enforcing:

A zero net beta constraint (to hedge liability risk).
A risk constraint (portfolio variance below a threshold).
Full investment with no short sales.

The model maximizes the portfolio’s expected return while ensuring that the net sensitivity to liability risk (the weighted sum of betas) is zero and that portfolio variance is controlled. The SLSQP method in SciPy’s minimize handles both equality and inequality constraints.

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

# Parameters for Hedged Return Portfolio
mu = np.array([0.08, 0.12, 0.02])   # Expected returns for three instruments
Sigma = np.array([[0.1, 0.03, 0.01],
                  [0.03, 0.2, 0.02],
                  [0.01, 0.02, 0.05]])  # Covariance matrix
beta = np.array([1.0, 0.5, -1.5])     # Beta sensitivities of each instrument
risk_threshold = 0.05                 # Maximum acceptable portfolio variance

# Objective function: maximize expected return.
# Since SciPy minimizes, we minimize the negative of expected return.
def hedged_return_objective(w):
    return -np.dot(mu, w)

# Constraint: portfolio must be beta neutral with respect to the liability factor.
def beta_constraint(w):
    return np.dot(beta, w)  # should equal 0

# Constraint: portfolio risk (variance) must be below the risk threshold.
def risk_constraint(w):
    risk = np.dot(w, np.dot(Sigma, w))
    return risk_threshold - risk  # inequality: risk <= threshold

# Constraints: full investment, zero beta, and risk constraint.
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},   # weights sum to 1
    {'type': 'eq', 'fun': beta_constraint},             # beta neutrality
    {'type': 'ineq', 'fun': risk_constraint}            # risk must be below threshold
]
# Bounds: no short selling.
bounds = [(0, 1) for _ in range(len(mu))]

# Initial guess for weights
w0 = np.array([1/3, 1/3, 1/3])

# Solve the optimization problem.
result = minimize(hedged_return_objective, w0, bounds=bounds, constraints=constraints)

print("Hedged Return Portfolio Optimal Weights:")
print(result.x)

# Presenting results using a DataFrame.
df_hedged = pd.DataFrame({
    'Instrument': ['Instrument 1', 'Instrument 2', 'Instrument 3'],
    'Weight': result.x
})
df_hedged


## Liability Driven Portfolio (LDP

The LDP approach focuses on matching the cash flows from bonds (or other fixed-income instruments) to the scheduled liabilities. Here, we minimize the sum of squared differences between the portfolio cash flows and the liabilities while ensuring full investment and nonnegative allocations.)

This example uses a least-squares approach to determine bond allocations that best match a predetermined liability schedule. The objective minimizes the squared error between the portfolio’s cash flows (obtained by multiplying the bond cash flow matrix by the allocation vector) and the liabilities, subject to full investment and nonnegativity constraints.

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

# Liability cash flows for 3 time periods (e.g., in millions)
liabilities = np.array([100, 150, 200])

# Cash flow matrix for 3 bonds over the same 3 periods.
# Each column corresponds to a bond; each row corresponds to a period.
cash_flows = np.array([
    [10, 20, 30],    # Period 1 cash flows
    [20, 30, 40],    # Period 2 cash flows
    [60,  90, 70]    # Period 3 cash flows (maturity payments)
])

# Objective function: minimize squared differences between portfolio cash flows and liabilities.
def ldp_objective(x):
    portfolio_cash_flows = cash_flows.dot(x)
    return np.sum((portfolio_cash_flows - liabilities) ** 2)

# Constraint: weights must sum to 1.
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
# Bounds: allocations must be between 0 and 1.
bounds = [(0, 1) for _ in range(cash_flows.shape[1])]

# Initial guess for allocations
x0 = np.array([1/3, 1/3, 1/3])

# Solve the optimization problem.
result = minimize(ldp_objective, x0, bounds=bounds, constraints=constraints)

print("Liability-Driven Portfolio (LDP) Optimal Allocations:")
print(result.x)

# Create a DataFrame to display the bond allocations.
df_ldp = pd.DataFrame({
    'Bond': ['Bond 1', 'Bond 2', 'Bond 3'],
    'Allocation': result.x
})

df_ldp


In [ ]:
10 * 0.55 + 20